In [1]:
import pandas as pd
import warnings
import re
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [2]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [3]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [4]:
download_and_extract(healthy_url, healthy_path)

In [5]:
download_and_extract(parkinson_url, parkinson_path)

In [6]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [7]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []
seq = 100

In [8]:
# create dataset with sequence length

# healthy signals
for filename in os.listdir(healthy_signals):
  if re.search('sigSp', filename) is not None:
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if re.search('sigSp', filename) is not None:
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1) 


In [9]:
len(X)

46901

In [10]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [11]:
X.shape

(46901, 100, 6)

In [12]:
y.shape

(46901,)

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)


In [15]:
model = keras.models.Sequential([
keras.layers.GRU(128, return_sequences=True, input_shape=(X_train.shape[1], 6)),
keras.layers.GRU(128),
keras.layers.Dense(1, activation="sigmoid")
])


In [16]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])


In [17]:
checkpoint_filepath = "model.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/50
1173/1173 [==============================] - 14s 12ms/step - loss: 0.1505 - accuracy: 0.7904 - val_loss: 0.1290 - val_accuracy: 0.8180
Epoch 2/50
1173/1173 [==============================] - 13s 11ms/step - loss: 0.1203 - accuracy: 0.8324 - val_loss: 0.1118 - val_accuracy: 0.8432
Epoch 3/50
1173/1173 [==============================] - 13s 11ms/step - loss: 0.1008 - accuracy: 0.8600 - val_loss: 0.0958 - val_accuracy: 0.8680
Epoch 4/50
1173/1173 [==============================] - 13s 11ms/step - loss: 0.0870 - accuracy: 0.8812 - val_loss: 0.0873 - val_accuracy: 0.8818
Epoch 5/50
1173/1173 [==============================] - 13s 11ms/step - loss: 0.0724 - accuracy: 0.9023 - val_loss: 0.0677 - val_accuracy: 0.9115
Epoch 6/50
1173/1173 [==============================] - 13s 11ms/step - loss: 0.0600 - accuracy: 0.9207 - val_loss: 0.0606 - val_accuracy: 0.9208
Epoch 7/50
1173/1173 [==============================] - 13s 11ms/step - loss: 0.0525 - accuracy: 0.9309 - val_loss: 0.0607 -

In [ ]:
# 100
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
# 3000
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
model.load_weights(checkpoint_filepath)
model.evaluate(X_valid, y_valid)

In [ ]:
!ls

In [ ]:
from google.colab import files
files.download('model.h5') 